## Test Emotion for Maya

In [2]:
import socket
import numpy as np
import pandas as pd
from DataAPI import MayaController
import itertools
import json
import os
from IPython.display import clear_output
from csv import writer

#### Open server

import maya.cmds as cmds

cmds.commandPort(n="localhost:12345")

cmds.commandPort(cl=True, n="localhost:12345")

In [3]:
controller = MayaController()

### Data Interpolation

In [3]:
df = pd.read_csv("../data/emotions.csv")
df

,Name,Brow_L_X,Brow_L_Y,Brow_R_X,Brow_R_Y,Mouth_M_X,Mouth_M_Y,Mouth_M_Lower,Mouth_M_Upper,Mouth_L_X,Mouth_L_Y,Mouth_R_X,Mouth_R_Y,Eye_L_S,Eue_R_S,V,A,D
0,Happy,0.025824,-0.2,0.025824,-0.20,0.40,0.0,0.0,0.0,3.000000e-01,1.00,3.000000e-01,1.00,0.0,0.0,1.000,0.735,0.772
1,Scared,0.500000,0.1,0.500000,0.10,0.20,-0.7,0.0,0.0,2.780000e-17,0.00,2.780000e-17,0.00,0.0,0.0,0.146,0.828,0.185
2,Angry,-0.600000,-0.4,-0.600000,-0.40,0.00,0.0,10.0,10.0,2.780000e-17,-0.20,2.780000e-17,-0.20,7.0,7.0,0.122,0.830,0.604
3,Curious,0.300000,0.0,0.300000,0.00,0.00,-0.3,0.0,0.0,2.780000e-17,0.20,2.780000e-17,0.20,0.0,0.0,0.635,0.600,0.483
4,Neutral,0.000000,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.000000e+00,0.00,0.000000e+00,0.00,0.0,0.0,0.469,0.184,0.357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Confused-Frustrated,-0.250000,-0.2,-0.150000,-0.20,0.05,0.0,7.5,2.5,-1.000000e-01,-0.05,3.000000e-01,-0.40,2.5,3.0,NaN,NaN,NaN
101,Confused-Tired,0.000000,-0.4,0.100000,-0.40,0.05,0.0,8.5,3.5,1.387779e-17,-0.10,2.000000e-01,-0.25,6.0,6.5,NaN,NaN,NaN
102,Shy-Frustrated,0.000000,-0.2,0.150000,-0.15,0.10,0.0,2.5,2.5,-1.000000e-01,0.30,5.000000e-02,-0.10,1.0,1.0,NaN,NaN,NaN
103,Shy-Tired,0.250000,-0.4,0.400000,-0.35,0.10,0.0,3.5,3.5,1.387779e-17,0.25,-5.000000e-02,0.05,4.5,4.5,NaN,NaN,NaN


In [14]:
def print_json(j):
    print(json.dumps(j, indent=4))

def StoreDataToCSV(data: dict, file_path: str, name:str="N/A", vad:list=None):
    '''
    Save the skeleton data to csv
    '''
    emo_lst = [name]
    for attr_dict in data.values():
        for attr in attr_dict.values():
            emo_lst.append(attr)
    if vad:
        emo_lst += vad

    with open(file_path, 'a') as write_obj:
        csv_writer = writer(write_obj)
        csv_writer.writerow(emo_lst)


In [15]:
def EmotionInterpolation(load_dir: str, save_dir: str, n:int =1):
    '''
    Interpolate the emotions and save both the screenshots and the facial attributes as dictionaries
    :param
        load_dir: loading directory of the facial poses
        save_dir: saving directory of the generated screenshots
        n: number of interpolations between poses 
    :return: list of interpolated emotion attributes
    '''
    # Get every poses
    poses_dir = os.listdir(load_dir)
    count = 1

    for strt_pose, end_pose in itertools.combinations(poses_dir, r=2):
        # Get the facial attributes of the starting and ending poses
        strt_dict = controller.GetFacialAttributesFromPose(os.path.join(os.path.join(load_dir, strt_pose), "pose.json"))
        end_dict = controller.GetFacialAttributesFromPose(os.path.join(os.path.join(load_dir, end_pose), "pose.json"))
        new_dict = strt_dict

        # Interpolate new emotions
        for i in range(n):
            for joint, attr_dict in strt_dict.items():
                for name, value in attr_dict.items():
                    diff = (end_dict[joint][name] - value) / (n + 1)
                    new_dict[joint][name] += diff
            
            index_str = strt_pose.split('.')[0]+'-'+end_pose.split('.')[0]
            # Apply Emotion and save a screenshot
            controller.SetFacialAttributes(new_dict)
            controller.ScreenShot(os.path.join(save_dir, index_str))
            # Save joint information to CSV
            StoreDataToCSV(new_dict, "../data/emotions.csv", index_str)

            # This is for debugging
            clear_output(wait=True)
            print(count)
            count += 1


In [18]:
EmotionInterpolation("../StudioLibrary/Poses/Face", "../data/Facial_img", 1)

91


In [20]:
controller.Close()